In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
image =cv2.imread('img.jfif')
from google.colab.patches import cv2_imshow
cv2_imshow(image)

In [ ]:
def coordinate(image,lineparameters):
  slope,intercept = lineparameters
  y1=image.shape[0]
  y2=int(y1*3/5)
  x1=int((y1-intercept)/slope)
  x2=int((y2-intercept)/slope)
  return np.array([x1,y1,x2,y2])

In [ ]:
def average_slope_intercept(image,lines):
  right_fit=[]
  left_fit=[]
  for line in lines:
    x1,y1,x2,y2 =line.reshape(4)
    parameters =np.polyfit((x1,x2),(y1,y2),1)
    slope=parameters[0]
    intercept = parameters[1]
    if slope<0:
      left_fit.append((slope,intercept))
    else:
      right_fit.append((slope,intercept))
  left_fit_average=np.average(left_fit,axis=0)
  right_fit_average=np.average(right_fit,axis=0)
  left_line=coordinate(image,left_fit_average)
  right_line=coordinate(image,right_fit_average)
  return np.array([left_line,right_line])

In [ ]:
def canny(image):
  gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
  blur=cv2.GaussianBlur(gray,(5,5),0)
  canny =cv2.Canny(blur,50,150)
  return(canny)

In [ ]:
def region(image):
  height=image.shape[0]
  polygons=np.array([[(50,height),(280,height),(150,40)]])
  m=np.zeros_like(image)
  cv2.fillPoly(m,polygons,255)
  mi=cv2.bitwise_and(image,m)
  return mi

In [ ]:
def lineform(image,lines):
  line_image = np.zeros_like(image)
  if lines is not None:
    for line in lines:
      x1,y1,x2,y2 = line.reshape(4)
      cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
  return line_image

In [ ]:
lane_image=np.copy(image)
canny=canny(lane_image)
mask=region(canny)
lines = cv2.HoughLinesP(mask,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
averaged_lines = average_slope_intercept(lane_image,lines)
line_image = lineform(lane_image,averaged_lines)
combined_image=cv2.addWeighted(lane_image,0.7,line_image,1,1)
from google.colab.patches import cv2_imshow
cv2_imshow(canny)
plt.imshow(canny)
plt.show()
mask=region(canny)
cv2_imshow(mask)
cv2_imshow(line_image)
cv2_imshow(combined_image)